<a href="https://colab.research.google.com/github/anaEver/DL/blob/main/Wisdom_Tweets_Author_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preperation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import io
import zipfile
from sklearn import preprocessing
!pip install emoji --upgrade
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from google.colab import files
from nltk.corpus import stopwords

#stop_words = stopwords.words('english')

import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=5f085c929cb3cdf64166c86d8b54661c9009f84ed891b3215075ea2d737f9fa2
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji
2.4.0


In [4]:
url = 'https://github.com/Hsankesara/The-Tweets-of-Wisdom/raw/master/tweets.csv'
#url = 'https://github.com/anaEver/DL/blob/main/export_dataframe.csv'
df = pd.read_csv(url)
df.tweet_content.size

31115

In [ ]:
df['tweet_content']=df['tweet_content'].str.lower()
df.head()

,author_name,created_at,handle,likes,retweets,tweet_content
0,Naval,2019-08-07 22:36:56,naval,7566,1498,"unresolved thoughts, prematurely pushed out of..."
1,Naval,2019-08-07 05:00:38,naval,21886,5984,the modern mind is overstimulated and the mode...
2,Naval,2019-08-07 04:52:33,naval,6462,1266,the lindy effect for startups:\n\nthe longer y...
3,Naval,2019-08-06 08:35:26,naval,466,61,@orangebook_ this was a good tweet.
4,Naval,2019-08-06 07:33:20,naval,3971,906,social media lowers the cost of raising &amp; ...


In [ ]:
import math
i = 0

for x in df.author_name.unique():
  if isinstance(x , str):
    
    y = df.loc[df['author_name'] == x, 'handle']
    #print(y.unique())
    if y.nunique()>1:
      print("*****************author:"+x)
      df.loc[df['author_name'] == x, 'handle'] = x
      print("handles: {}".format(y.nunique()))
      i = i + 1

for x in df.author_name:
  if isinstance(x , str):   
    i = i + 1
  elif math.isnan(x):
    print(x)
    df.loc[i, 'author_name'] = 'undefined'
    i = i + 1
    

nan


In [ ]:
import emoji
i = 0
for x in df.author_name:
  if isinstance(x, str):
    y = emoji.demojize(x)
  elif math.isnan(x):
    y = 'undefined'
  df.loc[i, 'author_name'] = y
  i = i + 1
print(df.handle.nunique())
print(df.author_name.nunique())

2782
2782


In [ ]:
df.to_csv (r'./tweets.csv', index = False, header=True)
%ls

anscombe.json*                mnist_test.csv         tweets.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md*


In [ ]:
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(df.handle.values)
np.unique(labels)

array([   0,    1,    2, ..., 2779, 2780, 2781])

In [74]:
df.tweet_content  = df.tweet_content.astype(str)
xtrain, xtest, ytrain, ytest = train_test_split(df.tweet_content.values, labels, 
  
 random_state=42, 
 test_size=0.2, shuffle=True)

# GloVe

In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2021-01-10 11:39:35--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2021-01-10 11:39:35--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2021-01-10 11:39:35--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [ ]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [ ]:
!ls
!pwd

anscombe.json		      glove.twitter.27B.200d.txt  mnist_test.csv
california_housing_test.csv   glove.twitter.27B.25d.txt   mnist_train_small.csv
california_housing_train.csv  glove.twitter.27B.50d.txt   README.md
glove.twitter.27B.100d.txt    glove.twitter.27B.zip	  tweets.csv
/content/sample_data


In [ ]:
print('Indexing word vectors.')

embeddings_index_100 = {}
f = open('glove.twitter.27B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_100[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_100))

Indexing word vectors.
Found 1193514 word vectors.


In [ ]:
!pip install --upgrade pip
!pip install -U -q pydrive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [64]:
import pickle
pickle.dump({'embeddings_index_100' : embeddings_index_100 } , open('drive/glove_100', 'wb'))

In [65]:
print('Indexing word vectors.')

embeddings_index_50 = {}
f = open('glove.twitter.27B.50d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_50[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_50))

Indexing word vectors.
Found 1193514 word vectors.


In [66]:
import pickle
pickle.dump({'embeddings_index_50' : embeddings_index_50 } , open('drive/glove_50', 'wb'))

In [75]:
VOCABULARY_SIZE = 2000
MAX_LENGTH = 60

tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer.fit_on_texts(list(xtrain) + list(xtest))

In [76]:
xtrain_sequence = tokenizer.texts_to_sequences(xtrain)
xtest_sequence = tokenizer.texts_to_sequences(xtest)

In [77]:
xtrain_padding = sequence.pad_sequences(xtrain_sequence, maxlen=MAX_LENGTH)
xtest_padding = sequence.pad_sequences(xtest_sequence, maxlen=MAX_LENGTH)
word_index = tokenizer.word_index

In [78]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in tqdm(word_index.items()):
  embedding_vector = embeddings_index_100.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

100%|██████████| 37226/37226 [00:00<00:00, 475277.34it/s]


# LSTM

In [88]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
 100,
 weights=[embedding_matrix],
 input_length=MAX_LENGTH,
 trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(50, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(2782))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [93]:
ytrain_encode = np_utils.to_categorical(ytrain, 2782)
ytest_encode = np_utils.to_categorical(ytest, 2782)

In [97]:

history = model.fit(xtrain_padding, y=ytrain_encode, batch_size=512, epochs=40, verbose=1, validation_data=(xtest_padding, ytest_encode))

Epoch 1/40
49/49 [==============================] - 49s 997ms/step - loss: 3.6780 - accuracy: 0.2234 - val_loss: 4.2246 - val_accuracy: 0.2439
Epoch 2/40
49/49 [==============================] - 48s 984ms/step - loss: 3.6815 - accuracy: 0.2237 - val_loss: 4.2316 - val_accuracy: 0.2462
Epoch 3/40
49/49 [==============================] - 48s 988ms/step - loss: 3.6614 - accuracy: 0.2274 - val_loss: 4.2294 - val_accuracy: 0.2433
Epoch 4/40
49/49 [==============================] - 48s 985ms/step - loss: 3.6631 - accuracy: 0.2272 - val_loss: 4.2270 - val_accuracy: 0.2470
Epoch 5/40
49/49 [==============================] - 48s 980ms/step - loss: 3.6532 - accuracy: 0.2284 - val_loss: 4.2454 - val_accuracy: 0.2446
Epoch 6/40
49/49 [==============================] - 48s 978ms/step - loss: 3.6470 - accuracy: 0.2259 - val_loss: 4.2465 - val_accuracy: 0.2452
Epoch 7/40
49/49 [==============================] - 48s 983ms/step - loss: 3.6481 - accuracy: 0.2289 - val_loss: 4.2476 - val_accuracy: 0.2457